In [ ]:
!cp /content/drive/MyDrive/Sprinklr_Internship/newinput.json ./input.json

cp: cannot stat '/content/drive/MyDrive/Sprinklr_Internship/newinput.json': No such file or directory


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!cp /content/drive/MyDrive/Sprinklr_Internship/output.json ./input.json

In [ ]:
!pip install pytesseract transformers datasets rouge-score nltk tensorboard py7zr accelerate evaluate --upgrade

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
# install git-fls for pushing model and logs to the hugging face hub
!sudo apt-get install git-lfs --yes

Reading package lists... Done
Building dependency tree       
Reading state information... Done
git-lfs is already the newest version (2.9.2-1).
0 upgraded, 0 newly installed, 0 to remove and 38 not upgraded.


In [ ]:
from huggingface_hub import notebook_login

notebook_login()

In [ ]:
from datasets import load_dataset

# Load dataset from the hub
dataset = load_dataset("samsum")

print(f"Train dataset size: {len(dataset['train'])}")
print(f"Test dataset size: {len(dataset['test'])}")

# Train dataset size: 14732
# Test dataset size: 819

Generating train split:   0%|          | 0/14732 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/819 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/818 [00:00<?, ? examples/s]

Dataset samsum downloaded and prepared to /root/.cache/huggingface/datasets/samsum/samsum/0.0.0/f1d7c6b7353e6de335d444e424dc002ef70d1277109031327bc9cc6af5d3d46e. Subsequent calls will reuse this data.


  0%|          | 0/3 [00:00<?, ?it/s]

Train dataset size: 14732
Test dataset size: 819


In [ ]:
from random import randrange        


sample = dataset['train'][randrange(len(dataset["train"]))]
print(f"dialogue: \n{sample['dialogue']}\n---------------")
print(f"summary: \n{sample['summary']}\n---------------")

dialogue: 
Caz: Car seems ok so I'm gong to shoot over to Coventry tomorrow afternoon to visit dad in the nursing home.. fingers crossed I'm not crying on the side of the road again xxx I cant wait to get away again xx
AJ:  Heres to another cruise
Caz: Til our next one lol
AJ: I'm hoping we will be having another very soon xxx
Caz: I nearly bought us one for April.. 7 nights to Portugal 780 pp
AJ: We can look for another over the weekend if you want xxx
Caz: Now we know.. we can sit and work out the sunny side of the ship before we book lol
AJ: I get some great bargain price emails xx
Caz: When do you have to put your holidays in or are they pretty flexible?
AJ: They are fairly flexible and there are only 4 of us controllers so next year is clear so far xxx
Caz: We will chat about it at the weekend then xxx
AJ: Ok babe, I should be off to bed soon for my 3am alarm 🙄xxx
Caz: Ok darling sleep good xxx
AJ: Night night babe xxx
---------------
summary: 
Caz is going to Coventry tomorrow af

In [ ]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

model_id="google/flan-t5-base"

# Load tokenizer of FLAN-t5-base
tokenizer = AutoTokenizer.from_pretrained(model_id)

In [ ]:
from datasets import load_dataset
from sklearn.model_selection import train_test_split

dataset = load_dataset("json", data_files="input.json")
desired_columns = ["context", "instruction", "response"]

for item in dataset['train']:
    item_copy = item.copy()
    item.clear()
    for col in desired_columns:
        if col in item_copy:
            item[col] = item_copy[col]
    for col in item_copy:
        if col not in desired_columns:
            item[col] = item_copy[col]

train_ds = dataset.copy()
test_ds = dataset.copy()

train_indices, test_indices = train_test_split(range(len(dataset['train'])), test_size=0.1, random_state=42)
train_ds_temp = dataset['train'].select(train_indices)
test_ds_temp = dataset['train'].select(test_indices)

train_ds['train'] = train_ds_temp
test_ds['test'] = test_ds_temp


Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Dataset json downloaded and prepared to /root/.cache/huggingface/datasets/json/default-3085fc2484376471/0.0.0/e347ab1c932092252e717ff3f949105a4dd28b27e842dd53157d2f72e276c2e4. Subsequent calls will reuse this data.


  0%|          | 0/1 [00:00<?, ?it/s]

In [ ]:
from datasets import load_dataset
from sklearn.model_selection import train_test_split

In [ ]:
dataset = load_dataset("json", data_files="input.json")

  0%|          | 0/1 [00:00<?, ?it/s]

In [ ]:
type(dataset)

datasets.dataset_dict.DatasetDict

In [ ]:
da = {}
da['sd'] = []
type(da)

dict

In [ ]:
dataset = load_dataset("json", data_files="input.json")
# desired_columns = ["context", "instruction", "response"]

# for split in dataset.keys():
#     for i in range(len(dataset[split])):
#         item = dataset[split][i]
#         item_copy = item.copy()
#         item.clear()
#         for col in desired_columns:
#             if col in item_copy:
#                 item[col] = item_copy[col]
#             else:
#                 item[col] = None

new_ds = {}
new_ds['train']={}

for split in dataset.keys():
    for i in range(len(dataset[split])):
    # for i in range(1):
        item = dataset[split][i]
        combined_data = f"{item['context']} {item['instruction']}"
        # print(combined_data)
        temp = {'data': combined_data, 'response': dataset[split][i]['response']}
        # print(temp)
        dataset[split][i].clear()
        # print(dataset[split][i])
        new_ds['train'][i]=temp




  0%|          | 0/1 [00:00<?, ?it/s]

In [ ]:
dataset = load_dataset("json", data_files="./newinput.json")

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

In [ ]:
dataset

DatasetDict({
    train: Dataset({
        features: ['instruction', 'response', 'context'],
        num_rows: 3279
    })
})

In [ ]:
import json
json_data = json.dumps(new_ds)
with open("./newinput.json", "w") as file:
    json.dump(json_data, file, indent=4)

In [ ]:
with open("./newinput", "r") as file

In [ ]:
train_indices, test_indices = train_test_split(range(len(new_ds['train'])), test_size=0.1, random_state=42)

train_ds = new_ds['train'].select(train_indices).map(lambda example: {k: example[k] for k in desired_columns})
test_ds = new_ds['train'].select(test_indices).map(lambda example: {k: example[k] for k in desired_columns})

dataset['train'] = train_ds
dataset['test'] = test_ds

In [ ]:
new_ds

In [ ]:
import json
with open("./newinput.json", 'w') as file:
    json.dump(new_ds, file, indent=4)

In [ ]:
dataset = load_dataset("json", data_files="./input.json")

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

ERROR:datasets.packaged_modules.json.json:Failed to read file '/content/input.json' with error <class 'pyarrow.lib.ArrowInvalid'>: JSON parse error: Missing a name for object member. in row 0


DatasetGenerationError: ignored

In [ ]:
dataset

In [ ]:
dataset['train'][0]

{'context': "The children of Bill 101 (les enfants de la loi 101) is the name given to the generation of children whose parents immigrated to Quebec, Canada after the adoption of the 1977 Charter of the French Language (Bill 101).\n\nOne of the Charter's articles stipulates that all children under 16 must receive their primary and secondary education in French schools, unless one of the child's parents has received most of their education in English, in Canada, or the child or the child's sibling has already received a substantial part of their education in English, in Canada. If a child qualifies for a certificate of eligibility for instruction in English under one of those criteria, then the certificate is permanent and can be passed on to future generations even if the child ends up attending French instead of English school. The permanent certificate can only be granted if one of the child's parents is a Canadian citizen. In addition, children of foreign temporary residents living 

In [ ]:
dataset

DatasetDict({
    train: Dataset({
        features: ['context', 'instruction', 'response'],
        num_rows: 2951
    })
    test: Dataset({
        features: ['context', 'instruction', 'response'],
        num_rows: 328
    })
})

In [ ]:
dataset['train'][0]

{'context': "The children of Bill 101 (les enfants de la loi 101) is the name given to the generation of children whose parents immigrated to Quebec, Canada after the adoption of the 1977 Charter of the French Language (Bill 101).\n\nOne of the Charter's articles stipulates that all children under 16 must receive their primary and secondary education in French schools, unless one of the child's parents has received most of their education in English, in Canada, or the child or the child's sibling has already received a substantial part of their education in English, in Canada. If a child qualifies for a certificate of eligibility for instruction in English under one of those criteria, then the certificate is permanent and can be passed on to future generations even if the child ends up attending French instead of English school. The permanent certificate can only be granted if one of the child's parents is a Canadian citizen. In addition, children of foreign temporary residents living 

In [ ]:
# from datasets import concatenate_datasets

# The maximum total input sequence length after tokenization. 
# Sequences longer than this will be truncated, sequences shorter will be padded.
tokenized_inputs = dataset["train"].map(lambda x: tokenizer(x["dialogue"], truncation=True), batched=True, remove_columns=["dialogue", "summary"])
max_source_length = max([len(x) for x in tokenized_inputs["input_ids"]])
print(f"Max source length: {max_source_length}")

# The maximum total sequence length for target text after tokenization. 
# Sequences longer than this will be truncated, sequences shorter will be padded."
tokenized_targets = concatenate_datasets([dataset["train"], dataset["test"]]).map(lambda x: tokenizer(x["summary"], truncation=True), batched=True, remove_columns=["dialogue", "summary"])
max_target_length = max([len(x) for x in tokenized_targets["input_ids"]])
print(f"Max target length: {max_target_length}")

KeyError: ignored

In [ ]:
def preprocess_function(sample,padding="max_length"):
    # add prefix to the input for t5
    inputs = ["summarize: " + item for item in sample["dialogue"]]

    # tokenize inputs
    model_inputs = tokenizer(inputs, max_length=max_source_length, padding=padding, truncation=True)

    # Tokenize targets with the `text_target` keyword argument
    labels = tokenizer(text_target=sample["summary"], max_length=max_target_length, padding=padding, truncation=True)

    # If we are padding here, replace all tokenizer.pad_token_id in the labels by -100 when we want to ignore
    # padding in the loss.
    if padding == "max_length":
        labels["input_ids"] = [
            [(l if l != tokenizer.pad_token_id else -100) for l in label] for label in labels["input_ids"]
        ]

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

tokenized_dataset = dataset.map(preprocess_function, batched=True, remove_columns=["dialogue", "summary", "id"])
print(f"Keys of tokenized dataset: {list(tokenized_dataset['train'].features)}")

Map:   0%|          | 0/14732 [00:00<?, ? examples/s]

Map:   0%|          | 0/819 [00:00<?, ? examples/s]

Map:   0%|          | 0/818 [00:00<?, ? examples/s]

Keys of tokenized dataset: ['input_ids', 'attention_mask', 'labels']


In [ ]:
from transformers import AutoModelForSeq2SeqLM

# huggingface hub model id
model_id="google/flan-t5-base"

# load model from the hub
model = AutoModelForSeq2SeqLM.from_pretrained(model_id)

In [ ]:
import evaluate
import nltk
import numpy as np
from nltk.tokenize import sent_tokenize
nltk.download("punkt")

# Metric
metric = evaluate.load("rouge")

# helper function to postprocess text
def postprocess_text(preds, labels):
    preds = [pred.strip() for pred in preds]
    labels = [label.strip() for label in labels]

    # rougeLSum expects newline after each sentence
    preds = ["\n".join(sent_tokenize(pred)) for pred in preds]
    labels = ["\n".join(sent_tokenize(label)) for label in labels]

    return preds, labels

def compute_metrics(eval_preds):
    preds, labels = eval_preds
    if isinstance(preds, tuple):
        preds = preds[0]
    decoded_preds = tokenizer.batch_decode(preds, skip_special_tokens=True)
    # Replace -100 in the labels as we can't decode them.
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    # Some simple post-processing
    decoded_preds, decoded_labels = postprocess_text(decoded_preds, decoded_labels)

    result = metric.compute(predictions=decoded_preds, references=decoded_labels, use_stemmer=True)
    result = {k: round(v * 100, 4) for k, v in result.items()}
    prediction_lens = [np.count_nonzero(pred != tokenizer.pad_token_id) for pred in preds]
    result["gen_len"] = np.mean(prediction_lens)
    return result

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [ ]:
from transformers import DataCollatorForSeq2Seq

# we want to ignore tokenizer pad token in the loss
label_pad_token_id = -100
# Data collator
data_collator = DataCollatorForSeq2Seq(
    tokenizer,
    model=model,
    label_pad_token_id=label_pad_token_id,
    pad_to_multiple_of=8
)

In [ ]:
from huggingface_hub import HfFolder
from transformers import Seq2SeqTrainer, Seq2SeqTrainingArguments

# Hugging Face repository id
repository_id = f"{model_id.split('/')[1]}-{dataset_id}"

# Define training args
training_args = Seq2SeqTrainingArguments(
    output_dir=repository_id,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    predict_with_generate=True,
    fp16=False, # Overflows with fp16
    learning_rate=5e-5,
    num_train_epochs=5,
    # logging & evaluation strategies
    logging_dir=f"{repository_id}/logs",
    logging_strategy="steps",
    logging_steps=500,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    save_total_limit=2,
    load_best_model_at_end=True,
    # metric_for_best_model="overall_f1",
    # push to hub parameters
    report_to="tensorboard",
    push_to_hub=False,
    hub_strategy="every_save",
    hub_model_id=repository_id,
    hub_token=HfFolder.get_token(),
)

# Create Trainer instance
trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=tokenized_dataset["train"],
    eval_dataset=tokenized_dataset["test"],
    compute_metrics=compute_metrics,
)

In [ ]:
# Start training
trainer.train()

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:407: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
You're using a T5TokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Epoch,Training Loss,Validation Loss,Rouge1,Rouge2,Rougel,Rougelsum,Gen Len
1,1.440300,1.382209,47.293600,23.820000,39.775600,43.514300,17.025641


In [ ]:
trainer.evaluate()

In [ ]:
# Save our tokenizer and create model card
tokenizer.save_pretrained(repository_id)
trainer.create_model_card()
# Push the results to the hub
trainer.push_to_hub()

In [ ]:
from transformers import pipeline
from random import randrange        

# load model and tokenizer from huggingface hub with pipeline
summarizer = pipeline("summarization", model="philschmid/flan-t5-base-samsum", device=0)

# select a random test sample
sample = dataset['test'][randrange(len(dataset["test"]))]
print(f"dialogue: \n{sample['dialogue']}\n---------------")

# summarize dialogue
res = summarizer(sample["dialogue"])

print(f"flan-t5-base summary:\n{res[0]['summary_text']}")

Your max_length is set to 200, but your input_length is only 191. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=95)


dialogue: 
Nathalie: have you thought about the holiday?
Pauline: me & tony are into greece really
Jacob: anywhere warm and sunny. greece cool
Anthony: greece is warm sunny and cheapish
Nathalie: i guess cob we ok w that
Jacob: sure thing
Pauline: so august as we said?
Jacob: thats the thing. we need to be back by aug 10
Anthony: what?? why??
Nathalie: sis wedding
Pauline: your lil sis getting married?!? lol
Jacob: she's not little. seen her tony?
Anthony: worth a look?
Nathalie: shut up assholes. shes my sister for fucks sake
Pauline: idiots
Jacob: come one just kidding. we love you
Anthony: we have no choice XD
---------------
flan-t5-base summary:
Nathalie, Pauline, Anthony and Anthony are going to Greece for a holiday in August. They need to be back by August 10 because of their sister's wedding.
